# Oasis FM Test Tool v0.6

This notebook allows example insurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Included

The following terminology is defined in OED. Please visit https://github.com/Simplitium/OED for more information.

* Standard direct insurance modelling 
    * Site (Coverages: Buildings, Other, Contents, BI, PD and All)
        * Ded, Limit
    * Cond (Coverages: All) A single condition per site only.
        * MinDed, Limit
    * Pol (Coverages: All)
        * Ded, MinDed, MaxDed, Limit
    * Layers - Multiple per account
        * Attachment, Limit, Participation
    * Deductible Types 0 and 2 (% TIV) only
    * Limit Types 0 and 2 (% TIV) only
    

## Not included
* Peril codes
* Type 1 terms (% loss)
* Code field


## Test cases

A selection of test cases can be found in the examples folder.


## Input files 

Files must be named the following:

* account.csv
* location.csv


## Validation Rules

* TBC

In [ ]:
# Notebook and Python setup
%config IPCompleter.greedy=True

# Standard Python libraries
import io
import json
import os

# 3rd party Python libraries
import jupyter_helper
import pandas as pd
import six
    
# Oasis imports
import oasislmf._data as static_data 
from oasislmf.model_preparation import oed
from oasislmf.utils.deterministic_loss import (
    generate_oasis_files,
    generate_losses,
)


In [ ]:
jupyter_helper.file_uploader('examples/uploaded')

In [ ]:
# Load the OED for a worked example, in this case FM Test Case 23.

# Note that only the currently used fields are shown unless show_all is set to True. 
oed_dir = os.path.abspath('examples_old/loc_SS')

# Account file
oed_account_file = os.path.join(oed_dir, "account.csv")
if not os.path.exists(oed_account_file):
    print("Path does not exist: {}".format(oed_account_file))
    exit(1)
account_df = pd.read_csv(oed_account_file)

# Location file
oed_location_file = os.path.join(oed_dir, "location.csv")
if not os.path.exists(oed_location_file):
    print("Path does not exist: {}".format(oed_location_file))
    exit(1)
location_df = pd.read_csv(oed_location_file)

In [ ]:
# View/edit the account data. 
account_grid = jupyter_helper.show_df(account_df)
account_grid

In [ ]:
# View/edit the location data.
location_grid = jupyter_helper.show_df(location_df)
location_grid

In [ ]:
# Pick up any edits in the grid before running the analysis
account_df = account_grid.get_changed_df()
location_df = location_grid.get_changed_df()

direct_oasis_files_dir = os.path.abspath('run_reinsurance')
account_df.to_csv(os.path.join(direct_oasis_files_dir, "account.csv"), index=False)
location_df.to_csv(os.path.join(direct_oasis_files_dir, "location.csv"), index=False)

data_fp = static_data.__path__._path[0]    
srcacctocan_trans_fp = os.path.join(data_fp, 'MappingMapToOED_CanAccA.xslt')
srcexptocan_trans_fp = os.path.join(data_fp, 'MappingMapToOED_CanLocA.xslt')
output_dir = os.path.join(direct_oasis_files_dir, "output")
(ri_layers, xref_descriptions) = generate_oasis_files(
    direct_oasis_files_dir, output_dir, 
    srcexptocan_trans_fp, srcacctocan_trans_fp)
loss_factor = 0.5
net_losses = generate_losses(
    output_dir, xref_descriptions, loss_percentage_of_tiv=loss_factor, ri_layers=ri_layers)

In [ ]:
# View the direct losses.
key = 'Direct'
net_losses[key]

## Optional Steps


In [ ]:
# Write outputs to CSV for download.
for (description, net_loss) in net_losses.items():
    filename = "output_{}.csv".format(description.replace(' ', '_'))
    net_loss.to_csv(filename, index=False)